In [3]:
import json
import random
from glob import glob
from unidecode import unidecode
from tqdm import tqdm

In [4]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

In [5]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian',
    'mistral',
    'terputus ialah 2021'
]

In [6]:
templates = [
"""
knowledge base is below.
---------------------
{context}
---------------------
Given the knowledge base and not prior knowledge, respond user question.
Question: {current_text}
""",
"""
Question: {current_text}, Given the knowledge base and not prior knowledge, respond user question.

knowledge base is below.
---------------------
{context}
---------------------
""",
"""
Question: {current_text}, Given the knowledge base and not prior knowledge, respond user question.

context: `{context}`
"""
]

In [9]:
rags = [
    '/home/husein/ssd3/soalan-augmentation/rag-multiturn-chaotic-part2.jsonl'
]

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

instructions = []
for f in rags:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            context = l['context']
            l = l['chat']
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    continue
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    continue

                role = roles[r['role']]
                
                if no == len(l) - 2 or no == 0 or (r['role'] == 'user' and random.random() > 0.7):
                    
                    current_text = random.choice(templates).format(context = context, current_text = current_text)
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 3:
                continue

            data = '\n'.join(inputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            instructions.append(d)

169403it [01:00, 2779.65it/s]


In [12]:
with open('prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl', 'w') as fopen:
    for d in instructions:
        fopen.write(f'{json.dumps(d)}\n')

In [13]:
!ls -lh prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl

-rw-r--r-- 1 husein husein 2.8G Feb  24 12:36 prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl


In [14]:
!wc -l prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl

167410 prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl


In [16]:
instructions[1]

{'prompt_input': None,
 'input': '<manusia>: \nQuestion: Saya harap anda tahu anda tidak sebenar, anda hanya sebuah program., Given the knowledge base and not prior knowledge, respond user question.\n\nknowledge base is below.\n---------------------\n9, Malacca: DLIS, FCSIT-UML, 2019 441 Wali M. H. 1978 .The role of school libraries Nigerian School Library Journal. Vol.1, no. 1: 35-38. Yousaf, M.I. 2005. Relationship of Performance of University Students with the Selected Variables of School Age. Unpublished PhD Thesis, University of Arid Agriculture. PRE-UNIVERSITY STUDIES (FOUNDATION STUDIES) 2 INTRODUCTION TO SEGi UNIVERSITY SEGi was established in 1977 as Systematic College in the heart of Kuala Lumpur offering professional qualifications. Since its founding over four decades ago, SEGi has undergone significant growth, strengthening the quality of its wide range of programmes from foundation to doctorate level. It is one of the most established private higher education institutions